In [9]:
from dotenv import load_dotenv
import os

load_dotenv()

from wikidatasearch.search import VectorSearch
from wikidatasearch.jina import JinaAIAPI

embedding_model = JinaAIAPI(os.environ['JINA_API_KEY'])
search = VectorSearch(os.environ, embedding_model=embedding_model)

ValueError: lid.176.bin cannot be opened for loading!

In [10]:
from abc import ABC, abstractmethod
from concurrent.futures import ThreadPoolExecutor, as_completed

from astrapy import DataAPIClient
from astrapy.api_options import APIOptions, TimeoutOptions
from stopwordsiso import stopwords
import requests
import re

datastax_tokens = os.environ

ASTRA_DB_APPLICATION_TOKEN = datastax_tokens['ASTRA_DB_APPLICATION_TOKEN']
ASTRA_DB_API_ENDPOINT = datastax_tokens["ASTRA_DB_API_ENDPOINT"]
ASTRA_DB_COLLECTION = datastax_tokens["ASTRA_DB_COLLECTION"]

timeout_options = TimeoutOptions(request_timeout_ms=100000)
api_options = APIOptions(timeout_options=timeout_options)

client = DataAPIClient(
    ASTRA_DB_APPLICATION_TOKEN,
    api_options=api_options
)
database0 = client.get_database(ASTRA_DB_API_ENDPOINT)
wikiDataCollection = database0.get_collection(ASTRA_DB_COLLECTION)

In [16]:
items = wikiDataCollection.find(filter={}, projection={'metadata.QID': 1, 'content': 0})
for item in items:
    print(item)
    break

APICommander about to raise from: [{'message': "Unsupported projection parameter: cannot exclude 'content' on inclusion projection", 'errorCode': 'UNSUPPORTED_PROJECTION_PARAM', 'id': '5daa1f9c-d663-4d2b-9aca-ad171c292a18', 'family': 'REQUEST', 'title': 'Unsupported projection parameter', 'scope': 'PROJECTION'}]


DataAPIResponseException: Unsupported projection parameter: Unsupported projection parameter: cannot exclude 'content' on inclusion projection (UNSUPPORTED_PROJECTION_PARAM)

In [8]:
import requests

r = requests.get('http://localhost:8000/item/query/?query=Hello&lang=en')
r.text

'{"detail":"Internal Server Error"}'

In [4]:
search.search("Germany")

[{'QID': 'Q183',
  'similarity_score': 0.8019527,
  'text': 'Germany, country in Central Europe, also known as de, GER, BRD, BR Deutschland, GFR, Deutschland, Federal Republic of Germany, Bundesrepublik Deutschland, DE. Attributes include: \n- instance of: sovereign state (end time: no value), country.\n- inception: 23 May 1949 (statement is subject of: West Germany).\n- has official name.\n- has native label.\n- has short name.\n- has pronunciation audio.\n- ethnic group: Germans (proportion: +0.863), Turks (proportion: +0.018), Poles (proportion: +0.01), Syrians (proportion: +0.01), Romanians (proportion: +0.01).\n- archives at: German Federal Archives.\n- drainage basin: Danube basin, Oder Basin, Rhine basin, Elbe Basin, Weser Basin.\n- life expectancy: +81.9 year (point in time: 2024).\n- named after: Germania (language of work or name: English, Russian, Greek, Albanian, Armenian, Bangla, Bulgarian, Esperanto, Georgian, Hausa, Hebrew, Hindi, Ido, Indonesian, Interlingua, Irish, Ita

In [30]:
filter = {"metadata.QID": "Q42"}
results = search.wikiDataCollection.find(
    filter,
    projection={"$vector": 1, "content": 0, "metadata": 0},
    limit=1
)
vector = next(results, {}).get('$vector')

In [39]:
relevant_items = search.wikiDataCollection.find(
    sort={"$vector": vector},
    limit=100,
    include_similarity=True,
    projection={"content": 1, "metadata": 1}
)

for item in relevant_items:
    print(item['metadata']['QID'], item['metadata']['Label'], item['$similarity'])
    print()

Q42 Douglas Adams 1.0

Q10268476 Douglas Howe Adams 0.8439025

Q22704934 Douglas H. Adams 0.8203218

Q732478 Thomas Adams 0.8157021

Q6242102 John Joseph Adams 0.8151555

Q105577325 John Adams 0.8149251

Q6218182 John Adams 0.81291986

Q1580430 Herbert Adams 0.8124901

Q7323638 Richard Adams 0.80972457

Q15206315 William Henry Davenport Adams 0.8096312

Q7341356 Robert Adams 0.8095787

Q1506819 George Jr. Adams 0.808528

Q5293950 Donald Adams 0.8069519

Q2082657 Edward Adams 0.8058032

Q6218177 John Adams 0.80576414

Q7407073 Sam Adams 0.80293334

Q10288941 George Adams 0.8029183

Q5211260 Dallas Adams 0.80278003

Q2646833 Alice Adams 0.8027684

Q7343665 Robert Dudley Adams 0.80198455

Q18670196 Adam Hughes 0.801438

Q972568 Nick Adams 0.80022335

Q5542247 George Matthew Adams 0.7999408

Q1122204 Michael J. Adams 0.79936993

Q75174 Charles Francis Adams, Sr. 0.7977937

Q2160347 John Adams 0.7973612

Q5536065 George Adams 0.7972263

Q185406 Samuel Hopkins Adams 0.7969191

Q434315 Russ A